In [1]:
# ── 설치(최초 1회) ─────────────────────────────────────────────
!pip -q install -U langchain langchain-openai langsmith


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 12.2 MB/s eta 0:00:00


In [ ]:
# 1) (선택) OpenAI 사용 시
import os, uuid
from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "llm_colab_ex_3"                 # 수업용 프로젝트명


In [3]:

# ── 임포트 ───────────────────────────────────────────────────
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableBranch, RunnablePassthrough

# ── LLM & 공통 파서 ──────────────────────────────────────────
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
parser = StrOutputParser()

# ── 안전 계산기(+, -, *, /, 괄호만 허용) ─────────────────────
import re
def safe_eval(expr: str) -> float:
    expr = expr.replace(" ", "")
    if not re.fullmatch(r"[0-9+\-*/().]+", expr):
        raise ValueError("허용되지 않는 문자가 포함되어 있습니다.")
    return eval(expr, {"__builtins__": {}}, {})

calc_chain = (
    {"expr": RunnableLambda(lambda x: str(x["input"]).split("calc:",1)[1].strip())}
    | RunnableLambda(lambda d: {"result": safe_eval(d["expr"])})
    | RunnableLambda(lambda d: f"계산 결과: {d['result']}")
)

# ── 일반 챗 체인 ──────────────────────────────────────────────
prompt_chat = ChatPromptTemplate.from_messages([
    ("system", "너는 인공지능 비서이다. 답변은 간결하게."),
    ("human", "{input}")
])
chat_chain = prompt_chat | llm | parser

# ── 라우터: 'calc:'로 시작하면 계산기로, 아니면 챗으로 ───────
router = RunnableBranch(
    (lambda x: str(x.get("input","")).lower().startswith("calc:"), calc_chain),
    chat_chain  # else
)

# ── 실행 (단건 + 배치) ───────────────────────────────────────
cfg = {"tags":["demo","routing","branch"], "run_name":"intent_router_demo"}

print("단건:")
print(router.invoke({"input":"calc: (23*7) + 10 / 2"}, config=cfg))

print("\n배치:")
outs = router.batch(
    [
        {"input":"오늘 LangSmith를 왜 쓰는지 간략하게 설명해줘"},
        {"input":"calc: 100*(2+3)"},
        {"input":"한국어로 RAG를 간단히 설명해줘"}
    ],
    config=cfg
)
for o in outs:
    print("-", o)


단건:
계산 결과: 166.0

배치:
- LangSmith는 자연어 처리(NLP) 모델을 개발하고 관리하는 데 도움을 주는 플랫폼으로, 모델의 성능을 향상시키고, 데이터 세트를 효율적으로 관리하며, 실험을 쉽게 수행할 수 있도록 지원합니다. 이를 통해 개발자와 연구자들이 더 나은 AI 솔루션을 구축할 수 있습니다.
- 계산 결과: 500
- RAG(Recall-Augmented Generation)는 정보 검색과 생성 모델을 결합한 방법입니다. 이 방식은 먼저 관련 정보를 검색한 후, 그 정보를 바탕으로 자연어를 생성하여 더 정확하고 풍부한 답변을 제공합니다. 주로 질문 응답 시스템이나 대화형 AI에서 사용됩니다.
